In [1]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [26]:
import findspark
findspark.init()


In [28]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("JupyterNotebook") \
    .getOrCreate()

spark


In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, avg, stddev, min, max

# Initialize Spark session with necessary configurations
spark = SparkSession.builder.appName("Credit Card Data Processing").config("spark.sql.shuffle.partitions", "8").config("spark.executor.memory", "2g").config("spark.local.dir", "/output").getOrCreate()

# Load the dataset
file_path = "creditcard.csv"  # Update with the actual path to your CSV file
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Display schema and initial data
data.printSchema()
data.show(5)

# Step 1: Data Partitioning
# Repartition data based on the 'Class' column to optimize distributed processing
data = data.repartition(4, col("Class"))

# Step 2: Data Transformation
# Add a new column categorizing transactions based on the amount
data = data.withColumn("Amount_Category", when(col("Amount") > 200, "High")
                                      .when((col("Amount") > 50) & (col("Amount") <= 200), "Medium")
                                      .otherwise("Low"))

# Step 3: Aggregation and Summaries
# Calculate aggregate statistics
summary = data.groupBy("Class").agg(
    count("*").alias("Transaction_Count"),
    avg("Amount").alias("Average_Amount"),
    stddev("Amount").alias("Stddev_Amount"),
    min("Amount").alias("Min_Amount"),
    max("Amount").alias("Max_Amount")
)

# Generate summary by Amount_Category
amount_summary = data.groupBy("Amount_Category").agg(
    count("*").alias("Transaction_Count"),
    avg("Amount").alias("Average_Amount")
)

# Step 4: Data Quality Report
# Check for null values
null_counts = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

# Check for duplicate rows
duplicate_count = data.count() - data.dropDuplicates().count()

# Display summaries and data quality metrics
print("Summary by Class:")
summary.show()

print("Summary by Amount Category:")
amount_summary.show()

print("Null Counts in Columns:")
null_counts.show()

print(f"Duplicate Rows Count: {duplicate_count}")

# Save the processed data to a directory
output_path = "/output/q3"
data.write.mode("overwrite").csv(output_path)

# Stop the Spark session
spark.stop()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

Py4JJavaError: An error occurred while calling o991.csv.
: java.io.IOException: Cannot run program "C:\Users\Lea\localapp\hadoop\spark-3.5.3-bin-hadoop3\bin\winutils.exe": CreateProcess error=2, The system cannot find the file specified
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:934)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.R

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, avg, stddev, min, max

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Credit Card Data Processing") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# Load the dataset
file_path = "creditcard.csv"  # Update with the actual path to your CSV file
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Display schema and initial data
data.printSchema()
data.show(5)

# Step 1: Data Partitioning
# Repartition data based on the 'Class' column to optimize distributed processing
data = data.repartition(4, col("Class"))

# Step 2: Data Transformation
# Add a new column categorizing transactions based on the amount
data = data.withColumn("Amount_Category", when(col("Amount") > 200, "High")
                                      .when((col("Amount") > 50) & (col("Amount") <= 200), "Medium")
                                      .otherwise("Low"))

# Step 3: Aggregation and Summaries
# Calculate aggregate statistics
summary = data.groupBy("Class").agg(
    count("*").alias("Transaction_Count"),
    avg("Amount").alias("Average_Amount"),
    stddev("Amount").alias("Stddev_Amount"),
    min("Amount").alias("Min_Amount"),
    max("Amount").alias("Max_Amount")
)

# Generate summary by Amount_Category
amount_summary = data.groupBy("Amount_Category").agg(
    count("*").alias("Transaction_Count"),
    avg("Amount").alias("Average_Amount")
)

# Step 4: Data Quality Report
# Check for null values
null_counts = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

# Check for duplicate rows
duplicate_count = data.count() - data.dropDuplicates().count()

# Display summaries and data quality metrics
print("Summary by Class:")
summary.show()

print("Summary by Amount Category:")
amount_summary.show()

print("Null Counts in Columns:")
null_counts.show()

print(f"Duplicate Rows Count: {duplicate_count}")

# Save the processed data to a directory
output_path = "output"
data.write.mode("overwrite").parquet(output_path)

# Stop the Spark session
spark.stop()


root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

Py4JJavaError: An error occurred while calling o482.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\Users\Lea\localapp\hadoop\spark-3.5.3-bin-hadoop3\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSy

In [34]:

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Jupyter PySpark Session") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

# Check the Spark Session
print(f"Spark version: {spark.version}")




Spark version: 3.5.3


In [36]:
pip install ipykernel

In [2]:
conda install anaconda-clean


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\Lea\Anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [38]:
from pyspark.sql import SparkSession

# Check or create SparkSession
spark = SparkSession.builder \
    .appName("LargeScaleDataProcessing") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()


In [40]:
from pyspark import SparkContext

# Stop any existing SparkContext
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()


In [14]:
pip install pyspark.sql

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyspark.sql (from versions: none)
ERROR: No matching distribution found for pyspark.sql


In [9]:
from pyspark.sql import SparkSession

In [10]:
conda install openjdk

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Lea\Anaconda3

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openjdk-11.0.13            |       h2bbff1b_0       301.3 MB
    ------------------------------------------------------------
                                           Total:       301.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/win-64::openjdk-11.0.13-h2bbff1b_0




openjdk-11.0.13      | 301.3 MB  |            |   0% 
openjdk-11.0.13      | 301.3 MB  |            |   0% 
openjdk-11.0.13      | 301.3 MB  |            |   0% 
openjdk-11.0.13      | 301.3 MB  |            |   1% 
openjdk-11.0.13      | 301.3 MB  | 1          |   1% 
openjdk-11.0.13      | 301.3 MB  | 1          |   1% 
openjdk-11.0.13      | 301.3 MB  | 1          |   2% 
openjdk-11.0.13



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [12]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Lea\Anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    backports.functools_lru_cache-1.6.4|     pyhd3eb1b0_0           9 KB
    backports.tempfile-1.0     |     pyhd3eb1b0_1          11 KB
    future-0.18.3              |   py37haa95532_0         702 KB
    ------------------------------------------------------------
                                           Total:         722 KB

The following packages will be UPDATED:

  backports.functoo~                               1.5-py_2 --> 1.6.4-pyhd3eb1b0_0
  future                                      0.17.1-py37_0 --> 0.18.3-py37haa95532_0

The following packages will be DOWNGRADED:

  backports.tempfile                               1.0-py_1 --> 1.0-pyhd3eb1b0_1




backports.functools_ | 9 KB      |



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
python script.py